# Get sequences to predict on

In [1]:
import seqdata as sd
import pandas as pd
import numpy as np

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
sdata = sd.open_zarr("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq/analysis/2023_12_11/eugene/deepstarr_windows.zarr")
sdata

<xarray.Dataset>
Dimensions:    (_sequence: 8436833)
Coordinates:
  * _sequence  (_sequence) object 'chr2L:0-249' ... 'chrM:19500-19517'
Data variables:
    seq        (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>

In [4]:
# Upper case all the sequences
sdata["seq"] = sdata["seq"].str.upper()
sdata

<xarray.Dataset>
Dimensions:    (_sequence: 8436833)
Coordinates:
  * _sequence  (_sequence) object 'chr2L:0-249' ... 'chrM:19500-19517'
Data variables:
    seq        (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>

In [5]:
from eugene import preprocess as pp

In [6]:
pp.ohe_seqs_sdata(sdata, alphabet="DNA")

In [7]:
sdata

<xarray.Dataset>
Dimensions:    (_sequence: 8436833, length: 249, _ohe: 4)
Coordinates:
  * _sequence  (_sequence) object 'chr2L:0-249' ... 'chrM:19500-19517'
Dimensions without coordinates: length, _ohe
Data variables:
    seq        (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>
    ohe_seq    (_sequence, length, _ohe) uint8 0 1 0 0 0 0 1 0 ... 0 0 0 0 0 0 0

# Load model 

In [8]:
from eugene import models
from eugene.models import zoo

In [9]:
arch = zoo.DeepSTARR(
    input_len=249,
    output_dim=2
)

In [10]:
model_ckpt = "/cellar/users/aklie/projects/ML4GLand/issues/41/tutorial_logs/DeepSTARR/Trial1/checkpoints/epoch=6-step=22001.ckpt"

In [11]:
model = models.SequenceModule.load_from_checkpoint(
    checkpoint_path=model_ckpt,
    arch=arch,
)
model.eval()

SequenceModule(
  (arch): DeepSTARR(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 246, kernel_size=(7,), stride=(1,), padding=same)
        (1): BatchNorm1d(246, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (4): Conv1d(246, 60, kernel_size=(3,), stride=(1,), padding=same)
        (5): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (8): Conv1d(60, 60, kernel_size=(5,), stride=(1,), padding=same)
        (9): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (12): Conv1d(60, 120, kernel_size=(3,), stride=(1,), padding=same)
 

# Make predictions on sequences

In [12]:
import torch

In [13]:
ohe_seqs = torch.tensor(sdata["ohe_seq"].transpose("_sequence", "_ohe", "length").values, dtype=torch.float32)

: 

: 

In [13]:
preds = model.predict(ohe_seqs)

: 

: 

# Write to bigwig

In [15]:
import pyBigWig

In [16]:
bw_path = "/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq/analysis/2023_12_11/eugene/test.bw"

In [17]:
# Write predictions to bigwig
bw = pyBigWig.open(bw_path, "w")
bw.addHeader(
    [
        (chrom, end - start),
    ]
)
bw.addEntries(
    [chrom] * len(starts),
    list(starts),
    ends=list(starts + 1),
    values=[float(x) for x in preds[:, 0]]
    
)
bw.close()

: 

: 

# DONE!

---

# Scratch

## What does the downloaded bw file look like?

In [17]:
downloaded_bw_path = "/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq/analysis/2023_12_11/GSE183936_S2_dev_STARRseq_merged.bw"

In [25]:
test_bw = pyBigWig.open("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq/analysis/2023_12_11/eugene/test.bw")

In [30]:
test_bw.chroms()
test_bw.header()

{'version': 4,
 'nLevels': 0,
 'nBasesCovered': 52501,
 'minVal': -1,
 'maxVal': 6,
 'sumData': -28225,
 'sumSquared': 45619}

In [28]:
np.array(test_bw.values(chrom, start+1000, end-1000))

RuntimeError: Invalid interval bounds!

In [20]:
# Read downloaded bigwig
downloaded_bw = pyBigWig.open(downloaded_bw_path)
downloaded_preds = np.array(downloaded_bw.values(chrom, start, end))

In [23]:
np.nanmax(downloaded_preds)

358.9930114746094